In [3]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re

In [ ]:
#get company name from mainpage
def get_company_name(soup):
    content_company_name = []
    for link in soup.find_all("a", {"class":"exhibitorName"}):
        text = link.get_text()
        text = text.strip()
        content_company_name.append(text)
    return content_company_name

In [ ]:
#find all internal links from mainpage
def get_link(soup):
    links = []
    for link in soup.find_all("a", {"class":"exhibitorName"}):
            text = link.get("href")
            text = text.strip()
            links.append(text)
    return links

In [ ]:
#get booth number from mainpage
def get_booth(soup):
    content_booth = []
    for link in soup.find_all("a", {"class":"boothLabel aa-mapIt"}):
        text = link.get_text()
        text = text.strip()
        content_booth.append(text)
    return content_booth

In [ ]:
#get external url from each individual company page
def get_website(soup):
    if soup.find_all("a", {"class":"aa-BoothContactUrl"}):
        for link in soup.find_all("a", {"class":"aa-BoothContactUrl"}):
            text = link.get_text()
            text = text.strip()
    else:
        text = "No Website"
    return text

In [ ]:
get_website(soup)

In [ ]:
soup

In [ ]:
url = "https://iss.a2zinc.net/LongBeach2020/Public/exhibitors.aspx?ID=20972" # url to scrape

f = open("token.txt", "r") # read token from local textfile for security reasons
access_token = f.read()
f.close()

payload = {'api_key': access_token, 'url': #using external API to circumvent blocking and captchas
url}
r = requests.get('http://api.scraperapi.com', params=payload) #scraping the mainpage

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup)

In [ ]:
#checking length of all items, should be equal count
print(len(get_company_name(soup)))
print(len(get_link(soup)))
print(len(get_booth(soup)))

In [ ]:
df = pd.DataFrame() # creating empty DF

In [ ]:
df["Company_Name"] = get_company_name(soup)

In [ ]:
df["Internal_link"] = get_link(soup)
df["Link"] = "https://iss.a2zinc.net/LongBeach2020/Public/" + df["Internal_link"]
df = df.drop("Internal_link", axis=1)

In [ ]:
df["Booth"] = get_booth(soup)

In [ ]:
internal_links = df["Link"].tolist()

In [ ]:
websites = []

for link in internal_links: # scraping each individual link in internal links list
    url = link
    payload = {'api_key': access_token, 'url': url}
    time.sleep(1)
    r = requests.get('http://api.scraperapi.com', params=payload)
    soup = BeautifulSoup(r.content, 'html.parser')
    websites.append(get_website(soup))

In [ ]:
len(websites)

In [ ]:
df["Website"] = websites

In [ ]:
#safe as csv
df.to_csv("ISS expo info.csv")

In [ ]:
#reopen csv to create DF again with first column as index
df = pd.read_csv("ISS expo info.csv", index_col=0)

In [ ]:
#Sanity check
df.head(10)

In [ ]:
#test scraping with last x urls
df_sample = df.iloc[-1:]
df_sample.head()

In [ ]:
#find email addresses on individual websites
urls = df_sample.iloc[:, 3] #getting urls from column 3
email_list = []

for url in urls:
    print("Parsing ", url)
    try:
        payload = {'api_key': access_token, 'url': url}
        time.sleep(1)
        r = requests.get('http://api.scraperapi.com', params=payload)
        soup = BeautifulSoup(r.content, 'html.parser')
    
        emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
        email_list.append(emails[0])
        print("Mail: ", emails[0])
        
    except:
        try:
            payload = {'api_key': access_token, 'url': url + "/contact/"}
            time.sleep(1)
            r = requests.get('http://api.scraperapi.com', params=payload)
            soup = BeautifulSoup(r.content, 'html.parser')

            emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
            email_list.append(emails[0])
            print("Mail: ", emails[0])
        except:
            try:
                payload = {'api_key': access_token, 'url': url + "/helpcenter/"}
                time.sleep(1)
                r = requests.get('http://api.scraperapi.com', params=payload)
                soup = BeautifulSoup(r.content, 'html.parser')

                emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
                email_list.append(emails[0])
                print("Mail: ", emails[0])
            except:
                try:
                    payload = {'api_key': access_token, 'url': url + "/contact/contact-us/"}
                    time.sleep(1)
                    r = requests.get('http://api.scraperapi.com', params=payload)
                    soup = BeautifulSoup(r.content, 'html.parser')

                    emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
                    email_list.append(emails[0])
                    print("Mail: ", emails[0])
                except:
                    try:
                        payload = {'api_key': access_token, 'url': url + "/contact/"}
                        time.sleep(1)
                        r = requests.get('http://api.scraperapi.com', params=payload)
                        soup = BeautifulSoup(r.content, 'html.parser')

                        emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
                        email_list.append(emails[0])
                        print("Mail: ", emails[0])
                    except:
                        try:
                            payload = {'api_key': access_token, 'url': url + "/contact-us/"}
                            time.sleep(1)
                            r = requests.get('http://api.scraperapi.com', params=payload)
                            soup = BeautifulSoup(r.content, 'html.parser')

                            emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
                            email_list.append(emails[0])
                        except:
                            try:
                                payload = {'api_key': access_token, 'url': url + "/contact-us"}
                                time.sleep(1)
                                r = requests.get('http://api.scraperapi.com', params=payload)
                                soup = BeautifulSoup(r.content, 'html.parser')

                                emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",str(soup))
                                email_list.append(emails[0])
                            except:
                                email_list.append("No E-Mail")
                                print("No Mail")


In [ ]:
#checking the emails that we scraped
email_list

In [ ]:
#Getting rid of incorrect support... emails
email_list = [i.replace('support@scraperapi.com',"No E-Mail") for i in email_list]
email_list

In [ ]:
#TODO: 
#-check all websites when enough api calls are available again
#-add email column to df

In [ ]:
#<3